In [15]:
import graphlab

In [16]:
graphlab.canvas.set_target('ipynb')

In [17]:
taps = graphlab.SFrame.read_csv("wow/6shcnktccn86m96hmcbng29s6t_taps.csv", verbose=False)

In [18]:
features = [
    "n_accel",
    
    "accel_x_mean",
    "accel_x_median",
    "accel_x_var",
    "accel_x_skewness",
    "accel_x_kurtosis",
    "accel_x_diff",
    
    "accel_y_mean",
    "accel_y_median",
    "accel_y_skewness",
    "accel_y_kurtosis",
    "accel_y_var",
    "accel_y_diff",
    
    "accel_z_mean",
    "accel_z_var",   
    "accel_z_median",
    "accel_z_skewness",
    "accel_z_kurtosis",
    "accel_z_diff",
    
    "n_gyro",
    
    "gyro_x_mean",
    "gyro_x_median",
    "gyro_x_var",
    "gyro_x_skewness",
    "gyro_x_kurtosis",
    "gyro_x_diff",
    
    "gyro_y_mean",
    "gyro_y_median",
    "gyro_y_var",
    "gyro_y_skewness",
    "gyro_y_kurtosis",
    "gyro_y_diff",
    
    "gyro_z_mean",
    "gyro_z_median",
    "gyro_z_var",
    "gyro_z_skewness",
    "gyro_z_kurtosis",
    "gyro_z_diff",
    
] 



In [19]:
#train, test = graphlab.SFrame().read_csv("taps.csv").random_split(0.8, seed=1)
train, test = taps.random_split(0.8, seed=1)
#test = taps # graphlab.SFrame().read_csv("taps.csv")

In [20]:
noise_model = graphlab.load_model("noise_model")
type_model = graphlab.load_model("type_model")
touch_model = graphlab.load_model("touch_model")
swipe_model = graphlab.load_model("swipe_model")

In [21]:
noise = test[test['noise'] == 1]
not_noise = test[test['noise'] == 0]

touches = not_noise[not_noise['type'] == "TOUCH"]
swipes = not_noise[not_noise['type'] == "SWIPE"]

# [Ceiling Analysis explained](https://d3c33hcgiwev3.cloudfront.net/19.4-ApplicationExamplePhotoOCR-CeilingAnalysisWhatPartOfThePipelineToWorkOnNext.6acb6550b22b11e49c064db6ead92550/full/540p/index.mp4?Expires=1488067200&Signature=Wlu5B9uRkREYKt0~6bVR~i6Tp5zN6k-uCRjFczQZKDu1dy6a1R55DLNWtknej1AbqtmZccxk8qDUiIJ7kT9U48llZxK1Y1BV2jqHR~iSkYK-pLmqZDCH-bDkEyMgDj-bdckYvqadcFpKj308QhIW8ihdsql-h9ehQx5DlDG5ERs_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A)

# Full model

In [22]:
predicted_not_noise = test[noise_model.predict(test) == 0]


predicted_touches = predicted_not_noise[type_model.predict(predicted_not_noise) == "TOUCH"]
predicted_swipes = predicted_not_noise[type_model.predict(predicted_not_noise) == "SWIPE"]
touch_evaluation = touch_model.evaluate(predicted_touches)
swipe_evaluation = swipe_model.evaluate(predicted_swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 55.1153796773 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
| BOTTOM-CENTER |  BOTTOM-CENTER  |  333  |
|     CENTER    |      CENTER     |  281  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  201  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  172  |
|  CENTER-LEFT  |   CENTER-LEFT   |  151  |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  100  |
|   TOP-RIGHT   |    TOP-RIGHT    |   60  |
|   TOP-CENTER  |    TOP-CENTER   |   56  |
|    TOP-LEFT   |     TOP-LEFT    |   55  |
|     NOISE     |  BOTTOM-CENTER  |   55  |
+---------------+-----------------+-------+
[86 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 78.5395763657 %
Swipe final f1 28.5039048813 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+----

# When providing real not_noise

In [23]:
predicted_touches = not_noise[type_model.predict(not_noise) == "TOUCH"]
predicted_swipes = not_noise[type_model.predict(not_noise) == "SWIPE"]
touch_evaluation = touch_model.evaluate(predicted_touches)
swipe_evaluation = swipe_model.evaluate(predicted_swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 62.9399609934 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
| BOTTOM-CENTER |  BOTTOM-CENTER  |  343  |
|     CENTER    |      CENTER     |  290  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  207  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  181  |
|  CENTER-LEFT  |   CENTER-LEFT   |  158  |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  103  |
|   TOP-RIGHT   |    TOP-RIGHT    |   60  |
|   TOP-CENTER  |    TOP-CENTER   |   58  |
|    TOP-LEFT   |     TOP-LEFT    |   58  |
|  TOP->BOTTOM  |      CENTER     |   10  |
+---------------+-----------------+-------+
[81 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 88.8482632541 %
Swipe final f1 31.9395574093 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+----

# When providing real swipes and touches

In [24]:
touch_evaluation = touch_model.evaluate(touches)
swipe_evaluation = swipe_model.evaluate(swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 92.9101671833 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
| BOTTOM-CENTER |  BOTTOM-CENTER  |  352  |
|     CENTER    |      CENTER     |  308  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  211  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  191  |
|  CENTER-LEFT  |   CENTER-LEFT   |  164  |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  106  |
|   TOP-RIGHT   |    TOP-RIGHT    |   62  |
|    TOP-LEFT   |     TOP-LEFT    |   61  |
|   TOP-CENTER  |    TOP-CENTER   |   58  |
|  CENTER-LEFT  |      CENTER     |   7   |
+---------------+-----------------+-------+
[53 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 93.6262376238 %
Swipe final f1 96.3959192927 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+----

In [25]:
noise_model.evaluate(test)

{'accuracy': 0.9154596511971623,
 'auc': 0.9712234198443471,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  141  |
 |      0       |        0        |  4164 |
 |      1       |        1        |  2030 |
 |      1       |        0        |  431  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.8765112262521589,
 'log_loss': 0.22170771877339696,
 'precision': 0.9350529709811147,
 'recall': 0.8248679398618448,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+------+
 | threshold |      fpr       |      tpr       |  p   |  n   |
 +-----------+----------------+----------------+------+------+
 |    0.0    |      1.0       |   

In [26]:
type_model.evaluate(not_noise)

{'accuracy': 0.9630662020905923,
 'auc': 0.9919892742342266,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |    TOUCH     |      SWIPE      |   67  |
 |    SWIPE     |      TOUCH      |   92  |
 |    SWIPE     |      SWIPE      |  2597 |
 |    TOUCH     |      TOUCH      |  1549 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9511820693890083,
 'log_loss': 0.11530351339483288,
 'precision': 0.9439366240097502,
 'recall': 0.9585396039603961,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+-----+------+------+
 | threshold |      fpr       | tpr |  p   |  n   |
 +-----------+----------------+-----+------+------+
 |    0.0    |      1.0       | 1.0 | 1616 | 2689 |
 |   1e-05   | 

In [27]:
swipe_model.evaluate(swipes)

{'accuracy': 0.9676459650427668,
 'auc': 0.9981127880306332,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | BOTTOM->TOP  |   TOP->BOTTOM   |   1   |
 | TOP->BOTTOM  |   LEFT->RIGHT   |   8   |
 | TOP->BOTTOM  |   BOTTOM->TOP   |   12  |
 | BOTTOM->TOP  |   LEFT->RIGHT   |   3   |
 | TOP->BOTTOM  |   RIGHT->LEFT   |   3   |
 | LEFT->RIGHT  |   LEFT->RIGHT   |  413  |
 | BOTTOM->TOP  |   BOTTOM->TOP   |  954  |
 | RIGHT->LEFT  |   TOP->BOTTOM   |   15  |
 | TOP->BOTTOM  |   TOP->BOTTOM   |  780  |
 | RIGHT->LEFT  |   RIGHT->LEFT   |  455  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.9639591929274108,
 'log_loss': 0.09626637715440942

In [28]:
touch_model.evaluate(touches)

{'accuracy': 0.9362623762376238,
 'auc': 0.9961251969460514,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 53
 
 Data:
 +---------------+-----------------+-------+
 |  target_label | predicted_label | count |
 +---------------+-----------------+-------+
 |  BOTTOM-RIGHT |   BOTTOM-LEFT   |   1   |
 |  CENTER-RIGHT |   CENTER-RIGHT  |  191  |
 |  CENTER-RIGHT |   BOTTOM-LEFT   |   4   |
 | BOTTOM-CENTER |   CENTER-RIGHT  |   1   |
 |     CENTER    |   BOTTOM-LEFT   |   2   |
 |    TOP-LEFT   |    TOP-RIGHT    |   1   |
 |  CENTER-RIGHT |  BOTTOM-CENTER  |   4   |
 |   TOP-CENTER  |   BOTTOM-LEFT   |   1   |
 |  CENTER-RIGHT |    TOP-RIGHT    |   1   |
 |   TOP-CENTER  |   CENTER-RIGHT  |   1   |
 +---------------+-----------------+-------+
 [53 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.9291016718329576,
 'log_loss': 0.205